In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [360]:
class Torch_Pad2d(object):

  @staticmethod
  def forward(x, pad_param):
    N, C, H, W = x.shape
    layer = nn.ZeroPad2d(pad_param)
    tx = x.detach()
    tx.requires_grad = True
    out = layer(tx)
    cache = (x, pad_param, tx, out, layer)
    return out, cache
  
  @staticmethod
  def backward(dout, cache):
    try:
      x, _, tx, out, layer = cache
      out.backward(dout)
      dx = tx.grad.detach()
    except RuntimeError:
      dx = torch.zeros_like(tx)
    return dx

In [366]:
torch.manual_seed(42)
x = torch.randn(1, 3, 2, 2)
print(x)
pad_param = (0, 1, 0, 1)

tensor([[[[ 0.3367,  0.1288],
          [ 0.2345,  0.2303]],

         [[-1.1229, -0.1863],
          [ 2.2082, -0.6380]],

         [[ 0.4617,  0.2674],
          [ 0.5349,  0.8094]]]])


In [368]:
out_torch, cache_torch = Torch_Pad2d.forward(x, pad_param)
print("out_torch", out_torch)
dout = torch.randn(out_torch.shape)
print("dout", dout)
dx = Torch_Pad2d.backward(dout, cache_torch)
print("dx", dx)

out_torch tensor([[[[ 0.3367,  0.1288,  0.0000],
          [ 0.2345,  0.2303,  0.0000],
          [ 0.0000,  0.0000,  0.0000]],

         [[-1.1229, -0.1863,  0.0000],
          [ 2.2082, -0.6380,  0.0000],
          [ 0.0000,  0.0000,  0.0000]],

         [[ 0.4617,  0.2674,  0.0000],
          [ 0.5349,  0.8094,  0.0000],
          [ 0.0000,  0.0000,  0.0000]]]], grad_fn=<ConstantPadNdBackward0>)
dout tensor([[[[ 0.5232,  0.3466, -0.1973],
          [-1.0546,  1.2780,  0.1453],
          [ 0.2311,  0.0087,  0.4263]],

         [[ 0.5750, -0.6417, -0.4976],
          [ 0.4747, -2.5095,  0.4880],
          [ 0.7846,  0.0286,  0.6408]],

         [[ 0.5832,  0.2191,  0.5526],
          [-0.1853,  0.7528,  0.4048],
          [ 0.1785,  0.2649,  1.2732]]]])
dx tensor([[[[ 0.5232,  0.3466],
          [-1.0546,  1.2780]],

         [[ 0.5750, -0.6417],
          [ 0.4747, -2.5095]],

         [[ 0.5832,  0.2191],
          [-0.1853,  0.7528]]]])


In [377]:
class Python_Pad2d(object):
    
    @staticmethod
    def forward(x, pad_param):
        new_shape = tuple(
            left + size + right
            for size, (left, right) in zip(x.shape, pad_param)
        )

        out = torch.zeros(new_shape)


        original_area_slice = tuple(
            slice(left, left + size)
            for size, (left, right) in zip(x.shape, pad_param)
        )
        out[original_area_slice] = x

        cache = (x, pad_param)

        return out, cache

    @staticmethod
    def backward(dout, cache):

        dx = None
        x, pad_param = cache
        N, C, H, W = x.shape

        dx = torch.zeros_like(x)

        for n in range(N):
            for f in range(C):          
                for height in range(H):
                    for width in range(W):
                        dx[n, f, height, width] = dout[n, f, height, width]

        return dx

In [378]:
pad_param = ((0, 0), (0, 0), (0, 1), (0, 1))

In [379]:
out, cache = Python_Pad2d.forward(x, pad_param)
print("out_torch", out)
print("dout", dout)
dx = Python_Pad2d.backward(dout, cache)
print("dx", dx)

out_torch tensor([[[[ 0.3367,  0.1288,  0.0000],
          [ 0.2345,  0.2303,  0.0000],
          [ 0.0000,  0.0000,  0.0000]],

         [[-1.1229, -0.1863,  0.0000],
          [ 2.2082, -0.6380,  0.0000],
          [ 0.0000,  0.0000,  0.0000]],

         [[ 0.4617,  0.2674,  0.0000],
          [ 0.5349,  0.8094,  0.0000],
          [ 0.0000,  0.0000,  0.0000]]]])
dout tensor([[[[ 0.5232,  0.3466, -0.1973],
          [-1.0546,  1.2780,  0.1453],
          [ 0.2311,  0.0087,  0.4263]],

         [[ 0.5750, -0.6417, -0.4976],
          [ 0.4747, -2.5095,  0.4880],
          [ 0.7846,  0.0286,  0.6408]],

         [[ 0.5832,  0.2191,  0.5526],
          [-0.1853,  0.7528,  0.4048],
          [ 0.1785,  0.2649,  1.2732]]]])
dx tensor([[[[ 0.5232,  0.3466],
          [-1.0546,  1.2780]],

         [[ 0.5750, -0.6417],
          [ 0.4747, -2.5095]],

         [[ 0.5832,  0.2191],
          [-0.1853,  0.7528]]]])


In [295]:
def _pad_simple(array, pad_width):
    new_shape = tuple(
        left + size + right
        for size, (left, right) in zip(array.shape, pad_width)
    )

    padded = np.empty(new_shape)
d
    if fill_value is not None:
        padded.fill(fill_value)

        original_area_slice = tuple(
            slice(left, left + size)
            for size, (left, right) in zip(array.shape, pad_width)
        )
        padded[original_area_slice] = array

        return padded, original_area_slice

In [262]:
k = torch.randn(2, 3, 2, 2)
pad_width = ((0, 0), (0, 0), (0, 1), (0, 1))

In [246]:
out_k = _pad_simple(k, pad_width, 0)

In [247]:
print(out_k[0].shape)
print(out_k)

(2, 3, 3, 3)
(array([[[[ 2.44171505e+00,  6.18615572e-01,  0.00000000e+00],
         [ 2.08841700e-03,  2.44440916e-01,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

        [[ 1.09291940e-01,  2.32490934e-01,  0.00000000e+00],
         [ 5.89264476e-01, -1.39765462e-01,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

        [[ 7.41355492e-01,  1.17004340e+00,  0.00000000e+00],
         [ 1.30424586e+00, -6.58016671e-01,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]],


       [[[-5.88565386e-01,  1.23458492e+00,  0.00000000e+00],
         [-6.46761769e-01, -1.44416355e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

        [[ 1.60366472e+00, -1.18892979e+00,  0.00000000e+00],
         [ 7.61895426e-01, -2.54439802e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

        [[-1.52376928e+00,  1.18383429e+00, 

In [362]:
class Python_Pad2d(object):
    
    @staticmethod
    def forward(x, pad_param):
        new_shape = tuple(
            left + size + right
            for size, (left, right) in zip(x.shape, pad_param)
        )

        out = torch.zeros(new_shape)


        original_area_slice = tuple(
            slice(left, left + size)
            for size, (left, right) in zip(x.shape, pad_param)
        )
        out[original_area_slice] = x

        cache = (x, pad_param)

        return out, cache

    # @staticmethod
    # def backward(dout, cache):

    #     dx = None
    #     x, pad_param = cache
    #     N, C, H, W = x.shape

    #     dx = torch.zeros_like(x)

    #     for n in range(N):
    #         for f in range(C):          
    #             for height in range(H):
    #                 for width in range(W):
    #                     if width < pad_param[0][0]:
    #                         ip_x = pad_param[0][0] * 2 - width
    #                     elif (width >= pad_param[0][0] and width < x[n, f, height, width] + pad_param[0][0]):
    #                         ip_x = width
    #                     else:
    #                         ip_x = (x[n, f, height, width])





    #     return dx

In [357]:
y = model(x)

In [359]:
print(y)

tensor([[[[ 2.5299e-01,  2.0211e-01, -3.3439e-01,  ..., -5.1062e-01,
           -1.8064e-01,  7.5705e-01],
          [-8.9768e-01,  1.6595e-01,  6.2740e-01,  ..., -2.5235e-01,
           -2.6290e-01, -8.7032e-01],
          [-4.2169e-02, -9.3448e-02,  7.5028e-01,  ..., -1.4735e-02,
           -4.0353e-02, -2.1801e-02],
          ...,
          [ 4.3464e-01, -1.0639e+00, -7.3677e-01,  ...,  4.5393e-01,
           -7.7566e-01,  8.5183e-02],
          [-1.9633e-01, -1.5287e-01,  4.6648e-01,  ..., -8.7419e-01,
           -7.5986e-01,  9.6901e-02],
          [ 4.8961e-01, -2.6689e-01, -1.4840e+00,  ..., -3.4830e-02,
            1.5728e+00,  1.1642e-01]],

         [[-2.5682e-01,  1.2796e-01, -2.7770e-01,  ...,  1.9998e-01,
            3.2381e-01,  3.3077e-01],
          [-8.2870e-02,  3.5884e-01, -1.6997e-01,  ..., -6.2296e-01,
            2.1655e-01,  1.0613e-01],
          [-3.8244e-01,  1.2105e+00,  2.2007e-01,  ..., -4.4228e-01,
           -1.0280e+00, -1.0241e-01],
          ...,
     